In [2]:
import tensorflow as tf

In [6]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, tf.Tensor): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def parse_single_sample(fea, matrital, income):
  data = {
        'fea' : _bytes_feature(tf.io.serialize_tensor(fea)),
        'matrital' : _int64_feature(matrital),
        'income': _int64_feature(income)
  }
  example = tf.train.Example(features=tf.train.Features(feature=data))

  return example

"""
# save as tfrecord to disk, or use `tf.data.Dataset.from_tensor_slices` in memory
with tf.io.TFRecordWriter("data/census/tfrecords/train.tfrecords") as writer:
    with open("data/census/train_data.csv", "r") as f:
        for line in f.readlines():
            fields = line.strip().split(",")
            matrital, income, fea = int(fields[0]), int(fields[1]), [float(x) for x in fields[2:]]
            example = parse_single_sample(fea, matrital, income)
            writer.write(example.SerializeToString())
"""

'\n# save as tfrecord to disk, or use `tf.data.Dataset.from_tensor_slices` in memory\nwith tf.io.TFRecordWriter("data/census/tfrecords/train.tfrecords") as writer:\n    with open("data/census/train_data.csv", "r") as f:\n        for line in f.readlines():\n            fields = line.strip().split(",")\n            matrital, income, fea = int(fields[0]), int(fields[1]), [float(x) for x in fields[2:]]\n            example = parse_single_sample(fea, matrital, income)\n            writer.write(example.SerializeToString())\n'

In [17]:
class MMOEDense(object):
    def __init__(self, nExperts, inputDim, expertDim):
        initializer = tf.initializers.truncated_normal(mean=0.0, stddev=1.0)
        # (nExperts, inputDim, expertDim)
        self.experts = tf.Variable(initializer(shape=(nExperts, inputDim, expertDim), dtype=tf.float32), name="experts")

    def forward(self, input):
        """
            @input: (batch, inputDim)
        """
        pass

bs = 4
inputDim = 2
expertDim = 2
nExperts = 2

model = MMOEDense(2, inputDim, expertDim)
#print(model.experts)

input = tf.random.normal((bs, inputDim))
# (batch, 1, 1, inputDim), second dim is for expert broadcast, third dim is for matmul
input = tf.expand_dims(tf.expand_dims(input, axis=1), axis=1)
#print(input)

# (batch, nExperts, 1, expertDim) -> (batch, nExperts, expertDim)
output = tf.squeeze(tf.matmul(input, model.experts), 2)

tf.Tensor(
[[[ 0.35172206 -0.90488565]
  [ 1.1093799   0.20515792]]

 [[ 1.0601685  -0.6509977 ]
  [ 3.4317236   0.8349423 ]]

 [[-0.4493051   1.5196171 ]
  [-1.4017922  -0.22415169]]

 [[-0.6490441  -1.4295893 ]
  [-2.1782324  -0.70180565]]], shape=(4, 2, 2), dtype=float32)
